In [ ]:
"""
    This script is for manual sratch assay with only image sets from two different times.
    The first set is obviously from time zero. The second set should be from the end of the experiment.
    Each image set should be in a different folder. The the address of each folder should be given to the functions,
    not the images themselvs.

"""

In [ ]:
# import necessary libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage import io
from skimage.filters import threshold_otsu

In [ ]:
def image_sets(path):
    """
    Reads all the images in a folder as grey images.
    
    Parameters
    ----------
    path: The address of the folder containing all the image. It mush have '/' 
    in the end of it.
    
    Returns
    ----------
    A list of all images read in grey scale 
        
    """
    return [io.imread(file, as_gray = True) for file in glob.glob(path + "*.*")]

In [ ]:
def entropy_img(images):
    """
    Reads all the images in a list and labels the areas with lower entropy.
    
    Parameters
    ----------
    images: The list of images read in grey scale provided by 
    the image_sets function.
    
    Returns
    ----------
    A list of all images with theor areas of low entropy labeled.
    """
    
    return [entropy(img, disk(10)) for img in images] 

In [ ]:
def threshold(entropy_images):
    """
    Reads all the images from a list and threshold them.
    
    Parameters
    ----------
    entropy_images: The list of images labeled by entropy function provided
    by the entropy_img function.
    
    Returns
    ----------
    A list of thresholded images.
    """
    
    return [threshold_otsu(ent_img) for ent_img in entropy_images]

In [ ]:
def gap_area(thresh_img, entropy_images):
    """
    Measures the gap areas in a list of images.
    
    Parameters
    ----------
    thresh_img: List of thresholded images provided by threshold function.
    entropy_images: List of images labeled by entropy function provided 
                    by entropy_img function.
                    
    Returns
    ----------
    A list containing the gap areas in pixels
    """
    
    return [np.sum((ent <= thr) == 1) for ent, thr in zip(entropy_images, thresh_img)]

In [ ]:
def gap_diff(time_first, time_last, concs):
    """
    Measures difference of gap areas bewteen time first (time 0) and time last.
    
    Parameters
    ----------
    time_first: List containing the gap areas of the images in time first (time 0).
    time_last: List containing the gap areas of the images in the time last.
    concs: List containing the concentrations used in the experiment.
    
    Returns
    ----------
    A dictionary whose keys are the concentraions and valuse are the difference between
    the gap areas between time first (time 0) and time last.
    """
    
    return {x: abs(y - z) for x, y, z in zip(concs, time_first, time_last)}

In [ ]:
def dot_plot(diff_dict, title, file_name):
    """
    Plot a dot plot from dictionary
    
    Parameters
    ----------
    diff_dict: Dictionary provdided by gap_diff function
    title: Title for the plot
    file_name: Name for saving the file. By default the format is png. 
    
    Returns
    ----------
    A plot whose y axis values are the values of the dictionary and x axis values are the values
    othe the dictionary.
    """
    
    normalized = [((num - min(diff_dict.values())) / (max(diff_dict.values()) - min(diff_dict.values())) + 1) for num in diff_dict.values()]
    plt.plot(diff_dict.keys(), normalized, "bo")
    plt.title(title)
    return plt.savefig(file_name)

In [ ]:
def bar_plot(diff_dict, title, file_name):
    """
    Plot a bar plot from dictionary
    
    Parameters
    ----------
    diff_dict: Dictionary provdided by gap_diff function
    title: Title for the plot
    file_name: Name for saving the file. By default the format is png. 
    
    Returns
    ----------
    A plot whose y axis values are the values of the dictionary and x axis values are the values
    othe the dictionary.
    """
    
    plot = pd.Series(diff_dict.values(), index = diff_dict.keys()).plot(kind = "bar", yerr = np.std([*diff_dict.values()]), title = title, rot = 0).get_figure()
    return plot.savefig(file_name)